# 循环神经网络从零开始实现

从头基于循环神经网络实现字符级语言模型。在时光机器数据集上进行训练

In [3]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

## 1. 独热编码
回想一下，在train_iter中，每个词元都表示为一个数字索引， 将这些索引直接输入神经网络可能会使学习变得困难。 我们通常将每个词元表示为更具表现力的特征向量。 最简单的表示称为独热编码（one-hot encoding）

简而言之，将每个索引映射为相互不同的单位向量：假设词表中不同次元的数目为N（即len（vocab）），词元索引的范围为0到N-1。如果词元的索引是整数$i$，那么我们将创建一个长度为$N$的全0向量，并将第$i$处的元素设置为1。索引为0和2的独热向量如下：

In [4]:
print(F.one_hot(torch.tensor([0, 2]), len(vocab)))
print(len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])
28


我们每次采样小批量数据形状是二维张量：（批量大小，时间步数）。one_hot函数将这样一个小批量数据转换为三维张量，张量的最后一个维度等于词表的大小（len（vocab））。

最终获得的形状为（时间步数，批量大小，词表大小）的输出。

In [5]:
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28).shape # 做转置的目的是

torch.Size([5, 2, 28])

## 2. 初始化模型参数
接下来，我们初始化循环神经网络模型的模型参数。 隐藏单元数num_hiddens是一个可调的超参数。 当训练语言模型时，输入和输出来自相同的词表。 因此，它们具有相同的维度，即词表的大小。

In [6]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size # 输入
    
    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01
    
    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = torch.normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

## 3. 循环神经网络模型